In [122]:
 import pandas as pd
 import numpy as np

In [123]:
delivery = pd.read_csv('/content/IPL_Ball_by_Ball_2008_2022.csv')
payer = pd.read_csv('/content/Player.csv')
player_captain = pd.read_csv('/content/Player_Match.csv')

In [124]:
delivery.head(1)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [125]:
payer.head(1)

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,Unnamed: 7
0,1,SC Ganguly,8-Jul-72,Left_Hand,Right-arm medium,India,0,NaN


In [126]:
player_captain.head()

,Match_Id,Player_Id,Team_Id,Is_Keeper,Is_Captain
0,335987,1,1,0,1
1,335987,2,1,0,0
2,335987,3,1,0,0
3,335987,4,1,0,0
4,335987,5,1,0,0


In [127]:
 tmp_df = payer.merge(player_captain,on='Player_Id')[['Player_Name','Match_Id','Is_Captain']]

In [128]:
delivery.merge(tmp_df,left_on=['ID','batter'],right_on=['Match_Id','Player_Name'],how='left').fillna(0)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,Player_Name,Match_Id,Is_Captain
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,0,0,0,0,0,0,0,0,0,Rajasthan Royals,0,0.0,0.0
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,0,0,0,Rajasthan Royals,0,0.0,0.0
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,0,1,0,1,0,0,0,0,0,Rajasthan Royals,0,0.0,0.0
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,0,0,0,0,0,0,0,0,0,Rajasthan Royals,0,0.0,0.0
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,0,0,0,0,0,0,0,0,0,Rajasthan Royals,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225949,335982,2,14,5,P Kumar,I Sharma,SB Joshi,legbyes,0,1,1,0,0,0,0,0,Royal Challengers Bangalore,0,0.0,0.0
225950,335982,2,14,6,SB Joshi,I Sharma,P Kumar,0,1,0,1,0,0,0,0,0,Royal Challengers Bangalore,0,0.0,0.0
225951,335982,2,14,7,P Kumar,I Sharma,SB Joshi,0,0,0,0,0,0,0,0,0,Royal Challengers Bangalore,0,0.0,0.0
225952,335982,2,15,1,SB Joshi,LR Shukla,P Kumar,wides,0,1,1,0,0,0,0,0,Royal Challengers Bangalore,0,0.0,0.0


In [129]:
runs = delivery.groupby(['ID','batter'])['batsman_run'].sum().reset_index()

In [130]:
ball =  delivery.groupby(['ID','batter'])['batsman_run'].count().reset_index()

In [131]:
sixes = delivery[delivery['batsman_run']==6].groupby(['ID','batter'])['batsman_run'].count().reset_index()

In [132]:
fours = delivery[delivery['batsman_run']==4].groupby(['ID','batter'])['batsman_run'].count().reset_index()

In [133]:
final_df = runs.merge(ball,on=['ID','batter'],suffixes=('_runs','_balls')).merge(fours,on=['ID','batter'],how='left').merge(sixes,on=['ID','batter'],how='left')

In [134]:
final_df.fillna(0,inplace=True)

In [135]:
final_df.head()

,ID,batter,batsman_run_runs,batsman_run_balls,batsman_run_x,batsman_run_y
0,335982,AA Noffke,9,12,1.0,0.0
1,335982,B Akhil,0,2,0.0,0.0
2,335982,BB McCullum,158,77,10.0,13.0
3,335982,CL White,6,10,0.0,0.0
4,335982,DJ Hussey,12,12,1.0,0.0


In [136]:
final_df.rename(columns={
    'batsman_run_runs':'runs',
    'batsman_run_balls':'balls',
    'batsman_run_x':'fours',
    'batsman_run_y':'sixes'
},inplace=True)

In [137]:
final_df['strike_rate'] = round(final_df['runs']/final_df['balls'] *100,2)

In [138]:
final_df.head()

,ID,batter,runs,balls,fours,sixes,strike_rate
0,335982,AA Noffke,9,12,1.0,0.0,75.00
1,335982,B Akhil,0,2,0.0,0.0,0.00
2,335982,BB McCullum,158,77,10.0,13.0,205.19
3,335982,CL White,6,10,0.0,0.0,60.00
4,335982,DJ Hussey,12,12,1.0,0.0,100.00


In [139]:
final_df = final_df.merge(tmp_df,left_on=['ID','batter'],right_on=['Match_Id','Player_Name'],how='left').drop(columns=['Player_Name','Match_Id']).fillna(0)

In [150]:
final_df.head()

,ID,batter,runs,balls,fours,sixes,strike_rate,Is_Captain
0,335982,AA Noffke,9,12,1.0,0.0,75.00,0.0
1,335982,B Akhil,0,2,0.0,0.0,0.00,0.0
2,335982,BB McCullum,158,77,10.0,13.0,205.19,0.0
3,335982,CL White,6,10,0.0,0.0,60.00,0.0
4,335982,DJ Hussey,12,12,1.0,0.0,100.00,0.0


In [160]:
def dream_11_points(x):
  points = 0
  points+=x['runs']*1
  points+=x['sixes']*6
  points+=x['fours']*4
  if x['runs'] >= 30:
    points+=4
  if x['runs'] >= 50 and x['runs'] < 75:
    points+=8
  if x['runs'] >= 75 and x['runs'] < 100:
    points+=8
  if x['runs'] >= 100:
    points+=16
  if x['runs'] == 0:
    points-=2
  if x['Is_Captain'] == 1:
    points*=2
  if x['balls'] >= 10:
    if x['strike_rate'] > 170:
      points+=6
    elif x['strike_rate'] > 150 and x['strike_rate'] <= 170:
      points+=4

    elif x['strike_rate'] > 130 and x['strike_rate'] <= 150:
      points+=2

    elif x['strike_rate'] > 60 and x['strike_rate'] <= 70:
      points+=-2

    elif x['strike_rate'] > 50 and x['strike_rate'] <= 60:
      points+=-4
    else:
      points+=-6

  return points


In [163]:
final_df['score']=final_df[['runs','sixes','fours','Is_Captain','strike_rate','balls']].apply(dream_11_points,axis=1)

In [164]:
final_df

,ID,batter,runs,balls,fours,sixes,strike_rate,Is_Captain,score
0,335982,AA Noffke,9,12,1.0,0.0,75.00,0.0,7.0
1,335982,B Akhil,0,2,0.0,0.0,0.00,0.0,-2.0
2,335982,BB McCullum,158,77,10.0,13.0,205.19,0.0,302.0
3,335982,CL White,6,10,0.0,0.0,60.00,0.0,2.0
4,335982,DJ Hussey,12,12,1.0,0.0,100.00,0.0,10.0
...,...,...,...,...,...,...,...,...,...
14224,1312200,SV Samson,14,11,2.0,0.0,127.27,0.0,16.0
14225,1312200,Shubman Gill,45,43,3.0,1.0,104.65,0.0,61.0
14226,1312200,TA Boult,11,7,0.0,1.0,157.14,0.0,17.0
14227,1312200,WP Saha,5,7,1.0,0.0,71.43,0.0,9.0


In [165]:
export_df = final_df[['ID','batter','score']]

In [166]:
export_df

,ID,batter,score
0,335982,AA Noffke,7.0
1,335982,B Akhil,-2.0
2,335982,BB McCullum,302.0
3,335982,CL White,2.0
4,335982,DJ Hussey,10.0
...,...,...,...
14224,1312200,SV Samson,16.0
14225,1312200,Shubman Gill,61.0
14226,1312200,TA Boult,17.0
14227,1312200,WP Saha,9.0
